In [3]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from datasets import Dataset
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from xgboost import XGBClassifier
import pandas as pd

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_metrics_and_confusion(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    f1 = f1_score(y_true, y_pred, average='binary')
    conf_matrix = confusion_matrix(y_true, y_pred)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "confusion_matrix": conf_matrix
    }

In [6]:
# Load the dataset
df = pd.read_csv("/Users/klesi/Documents/Modified_SQL_Dataset.csv")

# Set up Stratified K-Fold
k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Track metrics
codebert_scores = []
dnn_scores = []
xgb_scores = []

fold = 1
for train_index, test_index in skf.split(df['Query'], df['Label']):
    print(f"\n🧪 Fold {fold} -------------------------")
    train_df = df.iloc[train_index]
    test_df = df.iloc[test_index]

    # Convert to Hugging Face Dataset Format
    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    # Tokenization function
    def tokenize_function(examples):
        return tokenizer(examples["Query"], padding="max_length", truncation=True, max_length=128)

    tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
    model = RobertaForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels=2)

    train_dataset = train_dataset.map(tokenize_function, batched=True).remove_columns(["Query"]).rename_column("Label", "labels").with_format("torch")
    test_dataset = test_dataset.map(tokenize_function, batched=True).remove_columns(["Query"]).rename_column("Label", "labels").with_format("torch")

    training_args = TrainingArguments(
        output_dir=f"./results_fold_{fold}",
        evaluation_strategy="epoch",
        save_strategy="no",
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir=f"./logs_fold_{fold}",
        load_best_model_at_end=False
    )

    def compute_metrics(pred):
        logits, labels = pred
        predictions = np.argmax(logits, axis=1)
        precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
        acc = accuracy_score(labels, predictions)
        return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    predictions = trainer.predict(test_dataset)
    y_true = predictions.label_ids
    y_pred = np.argmax(predictions.predictions, axis=1)

    codebert_scores.append(evaluate_metrics_and_confusion(y_true, y_pred))

    # ----- BERT Embeddings for DNN & XGBoost -----
    bert_model = BertModel.from_pretrained('bert-base-uncased')
    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def get_bert_embeddings(queries, batch_size=32):
        embeddings = []
        for i in tqdm(range(0, len(queries), batch_size), desc="Extracting BERT embeddings"):
            tokens = bert_tokenizer(queries[i:i+batch_size], return_tensors='pt', padding=True, truncation=True, max_length=128)
            with torch.no_grad():
                outputs = bert_model(**tokens)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
            embeddings.append(batch_embeddings)
        return np.vstack(embeddings)

    train_embeddings = get_bert_embeddings(list(train_df['Query']))
    test_embeddings = get_bert_embeddings(list(test_df['Query']))

    # DNN
    dnn_model = Sequential([
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    dnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

    dnn_model.fit(train_embeddings, train_df['Label'], epochs=5, batch_size=32, verbose=0)
    y_pred_dnn = (dnn_model.predict(test_embeddings) > 0.5).astype("int32")
    dnn_scores.append(evaluate_metrics_and_confusion(test_df['Label'], y_pred_dnn))

    # XGBoost
    xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    xgb.fit(train_embeddings, train_df['Label'])
    y_pred_xgb = xgb.predict(test_embeddings)
    xgb_scores.append(evaluate_metrics_and_confusion(test_df['Label'], y_pred_xgb))

    fold += 1


🧪 Fold 1 -------------------------


/Users/klesi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/24735 [00:00<?, ? examples/s]

Map:   0%|          | 0/6184 [00:00<?, ? examples/s]

/Users/klesi/Library/Python/3.9/lib/python/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/kk/by7jm7pj16g5h4227vd0p03m0000gn/T/ipykernel_20121/1707763636.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

{'eval_loss': 0.00693671265617013, 'eval_accuracy': 0.9988678634966844, 'eval_f1': 0.998461876510657, 'eval_precision': 0.9986813186813187, 'eval_recall': 0.9982425307557118, 'eval_runtime': 72.4961, 'eval_samples_per_second': 85.287, 'eval_steps_per_second': 2.676, 'epoch': 3.0}
{'train_runtime': 4754.4852, 'train_samples_per_second': 15.608, 'train_steps_per_second': 0.488, 'train_loss': 0.010721478407215383, 'epoch': 3.0}


  0%|          | 0/194 [00:00<?, ?it/s]

Extracting BERT embeddings: 100%|██████████| 194/194 [01:25<00:00,  2.26it/s]


194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 366us/step


/Users/klesi/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [19:29:36] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [8]:
def print_average_scores(scores, model_name):
    print(f"\n🔎 Average Results for {model_name}")
    for metric in ["accuracy", "precision", "recall", "f1"]:
        avg = np.mean([fold[metric] for fold in scores])
        print(f"{metric.capitalize():<10}: {avg:.4f}")

print_average_scores(codebert_scores, "CodeBERT")
print_average_scores(dnn_scores, "DNN")
print_average_scores(xgb_scores, "XGBoost")


🔎 Average Results for CodeBERT
Accuracy  : 0.9989
Precision : 0.9992
Recall    : 0.9979
F1        : 0.9985

🔎 Average Results for DNN
Accuracy  : 0.9977
Precision : 0.9975
Recall    : 0.9963
F1        : 0.9969

🔎 Average Results for XGBoost
Accuracy  : 0.9969
Precision : 0.9971
Recall    : 0.9944
F1        : 0.9957
